## Постановка задачи
Загрузим данные, приведем их к числовым, заполним пропуски, нормализуем данные и оптимизируем память.

Разделим выборку на обучающую/проверочную в соотношении 80/20.

Сформируем параллельный ансамбль из логистической регрессии, SVM, случайного леса и LightGBM. Используем лучшие гиперпараметры, подобранные ранее. Итоговое решение рассчитаем на основании весов (вероятностей).

Проведем предсказание и проверим качество через каппа-метрику.

Данные:
* https://video.ittensive.com/machine-learning/prudential/train.csv.gz

Соревнование: https://www.kaggle.com/c/prudential-life-insurance-assessment/

### Подключение библиотек

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import cohen_kappa_score, confusion_matrix, make_scorer
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
import lightgbm as lgb
from sklearn import preprocessing

### Загрузка данных

In [2]:
data = pd.read_csv("https://video.ittensive.com/machine-learning/prudential/train.csv.gz")
print (data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59381 entries, 0 to 59380
Columns: 128 entries, Id to Response
dtypes: float64(18), int64(109), object(1)
memory usage: 58.0+ MB
None


### Предобработка данных

In [3]:
data["Product_Info_2_1"] = data["Product_Info_2"].str.slice(0, 1)
data["Product_Info_2_2"] = pd.to_numeric(data["Product_Info_2"].str.slice(1, 2))
data.drop(labels=["Product_Info_2"], axis=1, inplace=True)
for l in data["Product_Info_2_1"].unique():
    data["Product_Info_2_1" + l] = data["Product_Info_2_1"].isin([l]).astype("int8")
data.drop(labels=["Product_Info_2_1"], axis=1, inplace=True)
data.fillna(value=-1, inplace=True)
data["Response"] = data["Response"] - 1

### Набор столбцов для расчета

In [4]:
columns_groups = ["Insurance_History", "InsurеdInfo", "Medical_Keyword",
                  "Family_Hist", "Medical_History", "Product_Info"]
columns = ["Wt", "Ht", "Ins_Age", "BMI"]
for cg in columns_groups:
    columns.extend(data.columns[data.columns.str.startswith(cg)])
print (columns)

['Wt', 'Ht', 'Ins_Age', 'BMI', 'Insurance_History_1', 'Insurance_History_2', 'Insurance_History_3', 'Insurance_History_4', 'Insurance_History_5', 'Insurance_History_7', 'Insurance_History_8', 'Insurance_History_9', 'Medical_Keyword_1', 'Medical_Keyword_2', 'Medical_Keyword_3', 'Medical_Keyword_4', 'Medical_Keyword_5', 'Medical_Keyword_6', 'Medical_Keyword_7', 'Medical_Keyword_8', 'Medical_Keyword_9', 'Medical_Keyword_10', 'Medical_Keyword_11', 'Medical_Keyword_12', 'Medical_Keyword_13', 'Medical_Keyword_14', 'Medical_Keyword_15', 'Medical_Keyword_16', 'Medical_Keyword_17', 'Medical_Keyword_18', 'Medical_Keyword_19', 'Medical_Keyword_20', 'Medical_Keyword_21', 'Medical_Keyword_22', 'Medical_Keyword_23', 'Medical_Keyword_24', 'Medical_Keyword_25', 'Medical_Keyword_26', 'Medical_Keyword_27', 'Medical_Keyword_28', 'Medical_Keyword_29', 'Medical_Keyword_30', 'Medical_Keyword_31', 'Medical_Keyword_32', 'Medical_Keyword_33', 'Medical_Keyword_34', 'Medical_Keyword_35', 'Medical_Keyword_36', 'M

### Нормализация данных

In [5]:
scaler = preprocessing.StandardScaler()
data_transformed = pd.DataFrame(scaler.fit_transform(pd.DataFrame(data,
                                                     columns=columns)))
columns_transformed = data_transformed.columns
data_transformed["Response"] = data["Response"]

### Оптимизация памяти

In [6]:
def reduce_mem_usage (df):
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if str(col_type)[:5] == "float":
            c_min = df[col].min()
            c_max = df[col].max()
            if c_min > np.finfo("f2").min and c_max < np.finfo("f2").max:
                df[col] = df[col].astype(np.float16)
            elif c_min > np.finfo("f4").min and c_max < np.finfo("f4").max:
                df[col] = df[col].astype(np.float32)
            else:
                df[col] = df[col].astype(np.float64)
        elif str(col_type)[:3] == "int":
            c_min = df[col].min()
            c_max = df[col].max()
            if c_min > np.iinfo("i1").min and c_max < np.iinfo("i1").max:
                df[col] = df[col].astype(np.int8)
            elif c_min > np.iinfo("i2").min and c_max < np.iinfo("i2").max:
                df[col] = df[col].astype(np.int16)
            elif c_min > np.iinfo("i4").min and c_max < np.iinfo("i4").max:
                df[col] = df[col].astype(np.int32)
            elif c_min > np.iinfo("i8").min and c_max < np.iinfo("i8").max:
                df[col] = df[col].astype(np.int64)
        else:
            df[col] = df[col].astype("category")
    end_mem = df.memory_usage().sum() / 1024**2
    print('Потребление памяти меньше на', round(start_mem - end_mem, 2), 'Мб (минус', round(100 * (start_mem - end_mem) / start_mem, 1), '%)')
    return df

In [7]:
data_transformed = reduce_mem_usage(data_transformed)
print (data_transformed.info())

Потребление памяти меньше на 40.49 Мб (минус 75.1 %)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59381 entries, 0 to 59380
Columns: 119 entries, 0 to Response
dtypes: float16(118), int8(1)
memory usage: 13.4 MB
None


### Разделение данных
Преобразуем выборки в отдельные наборы данных

In [8]:
data_train, data_test = train_test_split(data_transformed,
                                         test_size=0.2)
data_train = pd.DataFrame(data_train)
data_test = pd.DataFrame(data_test)
print (data_train.head())

              0         1         2         3         4         5         6  \
16581 -0.748535 -0.710449  0.365479 -0.500488 -1.634766 -0.169434  0.862305   
32658 -0.630859 -1.200195 -0.997070 -0.017761  0.611816 -0.169434  0.862305   
13818 -0.748535 -0.710449  0.895020 -0.500488  0.611816 -0.169434 -1.159180   
5607  -0.513672 -0.955078  1.727539 -0.016434  0.611816 -0.169434 -1.159180   
54265  0.308838  1.249023  1.424805 -0.352295 -1.634766 -0.169434  0.862305   

              7         8         9  ...       109       110       111  \
16581 -1.013672  0.864746 -0.928711  ... -0.083679  0.441650 -0.149292   
32658 -1.013672  0.862305 -0.928711  ... -0.083679  0.441650 -0.149292   
13818  1.100586 -1.156250  1.130859  ... -0.083679  0.441650 -0.149292   
5607   1.100586 -1.156250  1.130859  ... -0.083679  0.441650 -0.149292   
54265 -1.013672  0.874512 -0.928711  ... -0.083679 -2.263672 -0.149292   

            112       113       114       115       116      117  Response  
165

### Построение базовых моделей

In [9]:
x = pd.DataFrame(data_train, columns=columns_transformed)

SVM

In [10]:
model_svm = SVC(kernel='linear', probability=True, max_iter=10000)
model_svm.fit(x, data_train["Response"])

c:\users\nikolay\appdata\local\programs\python\python37\lib\site-packages\sklearn\svm\_base.py:231: ConvergenceWarning: Solver terminated early (max_iter=10000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)


SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='linear',
    max_iter=10000, probability=True, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

Логистическая регрессия

In [11]:
model_logr = LogisticRegression(max_iter=1000)
model_logr.fit(x, data_train["Response"])

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=1000,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

Случайный лес

In [16]:
model_rf = RandomForestClassifier(random_state=17, n_estimators=76,
            max_depth=17, max_features=27, min_samples_leaf=20)
model_rf.fit(x, data_train["Response"])

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=17, max_features=27,
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=20, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=76,
                       n_jobs=None, oob_score=False, random_state=17, verbose=0,
                       warm_start=False)

LightGBM, используем multiclass

In [17]:
model_lgb = lgb.LGBMRegressor(random_state=17, max_depth=18,
            min_child_samples=18, num_leaves=75, n_estimators=1000,
            objective="multiclass", num_class=8)
model_lgb.fit(x, data_train["Response"])

c:\users\nikolay\appdata\local\programs\python\python37\lib\site-packages\lightgbm\engine.py:148: UserWarning: Found `num_boost_round` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


LGBMRegressor(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
              importance_type='split', learning_rate=0.1, max_depth=18,
              min_child_samples=18, min_child_weight=0.001, min_split_gain=0.0,
              n_estimators=100, n_jobs=-1, num_boost_round=1000, num_class=8,
              num_leaves=75, objective='multiclass', random_state=17,
              reg_alpha=0.0, reg_lambda=0.0, silent=True, subsample=1.0,
              subsample_for_bin=200000, subsample_freq=0)

### Расчет предказаний
Кроме непосредственно значений дополнительно посчитаем вероятности совпадения с тем или иным классом

In [18]:
x_test = pd.DataFrame(data_test, columns=columns_transformed)

In [19]:
data_test_svm_proba = pd.DataFrame(model_svm.predict_proba(x_test))

In [20]:
data_test_logr_proba = pd.DataFrame(model_logr.predict_proba(x_test))

In [21]:
data_test_rf_proba = pd.DataFrame(model_rf.predict_proba(x_test))

In [22]:
data_test_lgb = pd.DataFrame(model_lgb.predict(x_test))

Несколько вариантов ансамблей с голосованием (выбор класса выполняется для каждого кортежа по отдельности):
* "Мягкое" голосование (в том числе, с определенными весами): суммируются вероятности каждого класса среди всех оценок, выбирается наибольшее.
* "Жесткое" (мажоритарное) голосование: выбирается самый популярный класс среди моделей (число моделей должно быть нечетным).
* Отсечение: из вероятностей моделей выбирается только наиболее значимые, например, больше 0.3.
* Экспертное голосование: вес оценки эксперта зависит от кортежа данных и самого класса (например, если определенная модель предсказывает определенный класс точнее других).

Здесь используем "мягкое" голосование, для этого необходимо рассчитать вероятности всех класса для каждого кортежа данных.

In [23]:
def vote_class (x):
    a = np.argmax(x.values)
    return a

In [71]:
data_test_proba = data_test_svm_proba.copy()
for i in range(0, 8):
    data_test_proba[i] = 5*data_test_proba[i]
    data_test_proba[i] = data_test_proba[i] + 5*data_test_logr_proba[i]
    data_test_proba[i] = data_test_proba[i] + data_test_rf_proba[i]
    data_test_proba[i] = data_test_proba[i] + 12*data_test_lgb[i]
data_test_proba["target"] = data_test_proba.apply(vote_class, axis=1)

### Оценка ансамбля
Рассчитаем оценку взвешенного предсказания 4 моделей

Кластеризация дает 0.192, kNN(100) - 0.382, лог. регрессия - 0.512/0.496, SVM - 0.95, реш. дерево - 0.3, случайный лес - 0.487, XGBoost - 0.536, градиентный бустинг - 0.56, LightGBM - 0.569, CatBoost - 0.542

In [72]:
print ("Ансамбль классификации:",
      round(cohen_kappa_score(data_test_proba["target"],
                data_test["Response"], weights="quadratic"), 3))

Ансамбль классификации: 0.56


### Матрица неточностей

In [73]:
print ("Ансамбль классификации\n",
    confusion_matrix(data_test_proba["target"], data_test["Response"]))

Ансамбль классификации
 [[ 300  151   15   27   53  123   31   18]
 [ 172  329   20    8   99   87   25   21]
 [  22   23   74   13    0    1    0    0]
 [  32   22   56  182    0    2    1    4]
 [  88  142   17    0  568   85    8    5]
 [ 259  266   19   21  227 1255  324  122]
 [ 103   96    1    5   46  262  606  123]
 [ 250  232    3   34  109  491  596 3603]]


### Само-проверка
Проверим, насколько ансамбль хорошо предсказывает обучающие данные

In [74]:
data_copy = data_train.copy()
x_copy = pd.DataFrame(data_copy, columns=columns_transformed)

In [75]:
data_copy_svm = pd.DataFrame(model_svm.predict_proba(x_copy))

In [76]:
data_copy_logr = pd.DataFrame(model_logr.predict_proba(x_copy))

In [77]:
data_copy_rf = pd.DataFrame(model_rf.predict_proba(x_copy))

In [78]:
data_copy_lgb = pd.DataFrame(model_lgb.predict(x_copy))

In [79]:
for i in range(0, 8):
    data_copy_svm[i] = 5*data_copy_svm[i]
    data_copy_svm[i] = data_copy_svm[i] + 5*data_copy_logr[i]
    data_copy_svm[i] = data_copy_svm[i] + data_copy_rf[i]
    data_copy_svm[i] = data_copy_svm[i] + 12*data_copy_lgb[i]
target = data_copy_svm.apply(vote_class, axis=1)

In [80]:
print ("Результат:",
      round(cohen_kappa_score(target,
                data_copy["Response"], weights="quadratic"), 3))

Результат: 0.964


In [82]:
print (confusion_matrix(target, data_copy["Response"]))

[[ 4741     0     0     0     0     0     0     0]
 [    1  5041     0     0     0     0     0     0]
 [    0     0   808     0     0     0     0     0]
 [    0     0     0  1138     0     0     0     0]
 [    3     6     0     0  4289     0     0     0]
 [   38    46     0     0     8  8476     6     2]
 [   13    13     0     0     1    15  5889     1]
 [  185   185     0     0    32   436   541 15590]]
